### SpaTalk

#### Breast Cancer Data for Louvain Clusters 1-4 / 4-1, 1-5 / 5-1, 1-1, 4-4 and 5-5 with stLearn LR database (connectomeDB)

In [5]:
################################### Loading the Libraries ###########################################################

library(SpaTalk)
library(tidyverse)
library(SeuratData)
library(Seurat)
library(hdf5r)
library(data.table)
library(SeuratDisk)

################################### Reading the ST, SC and Leiden Cluster Data ################################################

# Replace the LR pairs with stLearn
lrpairs <- fread("/home/uqomulay/90days/stLearn_rebuttal/SpaTalk/lrpairs.txt", sep="\t")
lrpairs <- lrpairs[,c("Ligand gene symbol","Receptor gene symbol")]
colnames(lrpairs)[c(1,2)] <- c("ligand","receptor")
lrpairs$species <- rep("Human",2293)

# Read the data - block1
mat_bc_seurat_obj <- Load10X_Spatial(data.dir="/afm01/UQ/Q1851/Jon/Human_Breast_Cancer_Block_A_Section_1/")
bc_seurat_obj <- t(as.matrix(GetAssayData(mat_bc_seurat_obj)))

# Selecting high variance genes
variances <- apply(X=bc_seurat_obj, MARGIN=2, FUN=var)
sorted <- sort(variances, decreasing=TRUE, index.return=TRUE)$ix[1:10000] 
bc_seurat_obj <- t(bc_seurat_obj[, sorted])

# Checking LR pairs in our data
lrpairs = lrpairs[lrpairs$ligand %in% rownames(bc_seurat_obj) & lrpairs$receptor %in% rownames(bc_seurat_obj)]

# Reading leiden clustering data
label_transfer_bc <- fread("/home/uqomulay/90days/stLearn_rebuttal/bc_leiden_cluster.csv")
label_transfer_bc$predicted.id <- as.character(label_transfer_bc$cell_type)

# Reading label transfer data
#label_transfer_bc <- fread("/home/uqomulay/90days/stLearn_rebuttal/block1/label_transfer_bc.csv")

label_transfer_bc <- label_transfer_bc %>% remove_rownames %>% column_to_rownames(var="V1")
label_transfer_bc <- label_transfer_bc[row.names(label_transfer_bc) %in% colnames(bc_seurat_obj),] 
bc_seurat_obj <- subset(bc_seurat_obj, cell = rownames(label_transfer_bc))

# Data Pre-processing - checing the barcodes
bc_seurat_obj_metadata <- merge(mat_bc_seurat_obj@images$slice1@coordinates,label_transfer_bc,by="row.names",all=FALSE)
bc_seurat_obj_metadata <- bc_seurat_obj_metadata %>% remove_rownames %>% column_to_rownames(var="Row.names")
colnames(bc_seurat_obj_metadata)[c(4,5)] <- c('x','y')
bc_seurat_obj_metadata <- bc_seurat_obj_metadata[,c("x","y","predicted.id")]
bc_seurat_obj_metadata$cell <- row.names(bc_seurat_obj_metadata)
bc_seurat_obj_metadata <- bc_seurat_obj_metadata[c("cell","x","y","predicted.id")]
colnames(bc_seurat_obj_metadata)[4] <- "celltype"

# Reading single cell data
breast_sc <- LoadH5Seurat("/afm01/UQ/Q1851/Brad/breast_sc.h5Seurat")
bc_seurat_obj <- bc_seurat_obj[,colnames(bc_seurat_obj) %in% rownames(bc_seurat_obj_metadata)]

################################### Creating SpaTalk object, and run dec_cci ################################################

################################### Generate pseudo spot data ###############################################################

# create SpaTalk data
obj <- createSpaTalk(st_data = bc_seurat_obj,
                     st_meta = bc_seurat_obj_metadata[,-4],
                     species = "Human",
                     if_st_is_sc = T,
                     spot_max_cell = 10,
                     celltype = bc_seurat_obj_metadata$celltype)


#Find LR Path and Decomposing Cell-type
obj <- find_lr_path(object = obj, lrpairs = lrpairs, pathways = pathways)
try(obj <- dec_cci(object=obj, celltype_sender="1", celltype_receiver="4", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="1", celltype_receiver="5", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="5", celltype_receiver="1", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="4", celltype_receiver="1", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="1", celltype_receiver="1", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="4", celltype_receiver="4", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="5", celltype_receiver="5", use_n_cores = 12, min_pairs = 3))

write.csv(obj@lrpair,"/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/LR_connectomeDB_leiden_cluster.csv")
write.csv(obj@tf,"/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/TF_connectomeDB_leiden_cluster.csv")
saveRDS(obj@cellpair, file = "/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/CellPair_connectomeDB_leiden_cluster.rds")

Validating h5Seurat file

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Initializing RNA with data

Adding counts for RNA

Adding scale.data for RNA

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”
Adding feature-level metadata for RNA

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding command information

Adding cell-level metadata

Adding miscellaneous information

Adding tool-specific results



Checking input data 
Begin to filter lrpairs and pathways 
***Done*** 
Begin to find LR pairs 
Error in dec_cci(object = obj, celltype_sender = "1", celltype_receiver = "5",  : 
  Cell pairs found between 1 and 5 less than min_pairs!
Error in dec_cci(object = obj, celltype_sender = "5", celltype_receiver = "1",  : 
  Cell pairs found between 5 and 1 less than min_pairs!
Begin to find LR pairs 
Begin to find LR pairs 
Begin to find LR pairs 
Error in dec_cci(object = obj, celltype_sender = "4", celltype_receiver = "4",  : 
  No LR pairs found between 4 and 4
Begin to find LR pairs 


#### SpaTalk for LabelTransfer Clusters with stLearn LR database (connectomeDB)

In [1]:
################################### Loading the Libraries ###################################################

library(SpaTalk)
library(tidyverse)
library(SeuratData)
library(Seurat)
library(hdf5r)
library(data.table)
library(SeuratDisk)

################################### Reading the ST, SC and Leiden Cluster Data ################################################

# Replace the LR pairs with stLearn
lrpairs <- fread("/home/uqomulay/90days/stLearn_rebuttal/SpaTalk/lrpairs.txt", sep="\t")
lrpairs <- lrpairs[,c("Ligand gene symbol","Receptor gene symbol")]
colnames(lrpairs)[c(1,2)] <- c("ligand","receptor")
lrpairs$species <- rep("Human",2293)

# Read the data - block1
mat_bc_seurat_obj <- Load10X_Spatial(data.dir="/afm01/UQ/Q1851/Jon/Human_Breast_Cancer_Block_A_Section_1/")
bc_seurat_obj <- t(as.matrix(GetAssayData(mat_bc_seurat_obj)))

# Selecting high variance genes
variances <- apply(X=bc_seurat_obj, MARGIN=2, FUN=var)
sorted <- sort(variances, decreasing=TRUE, index.return=TRUE)$ix[1:10000] 
bc_seurat_obj <- t(bc_seurat_obj[, sorted])

# Checking LR pairs in our data
lrpairs = lrpairs[lrpairs$ligand %in% rownames(bc_seurat_obj) & lrpairs$receptor %in% rownames(bc_seurat_obj)]

# Reading leiden clustering data
#label_transfer_bc <- fread("/home/uqomulay/90days/stLearn_rebuttal/bc_leiden_cluster.csv")
#label_transfer_bc$predicted.id <- as.character(label_transfer_bc$cell_type)

# Reading label transfer data
label_transfer_bc <- fread("/home/uqomulay/90days/stLearn_rebuttal/block1/label_transfer_bc.csv")

label_transfer_bc <- label_transfer_bc %>% remove_rownames %>% column_to_rownames(var="V1")
label_transfer_bc <- label_transfer_bc[row.names(label_transfer_bc) %in% colnames(bc_seurat_obj),] 
bc_seurat_obj <- subset(bc_seurat_obj, cell = rownames(label_transfer_bc))

# Data Pre-processing - checing the barcodes
bc_seurat_obj_metadata <- merge(mat_bc_seurat_obj@images$slice1@coordinates,label_transfer_bc,by="row.names",all=FALSE)
bc_seurat_obj_metadata <- bc_seurat_obj_metadata %>% remove_rownames %>% column_to_rownames(var="Row.names")
colnames(bc_seurat_obj_metadata)[c(4,5)] <- c('x','y')
bc_seurat_obj_metadata <- bc_seurat_obj_metadata[,c("x","y","predicted.id")]
bc_seurat_obj_metadata$cell <- row.names(bc_seurat_obj_metadata)
bc_seurat_obj_metadata <- bc_seurat_obj_metadata[c("cell","x","y","predicted.id")]
colnames(bc_seurat_obj_metadata)[4] <- "celltype"

# Reading single cell data
breast_sc <- LoadH5Seurat("/afm01/UQ/Q1851/Brad/breast_sc.h5Seurat")
bc_seurat_obj <- bc_seurat_obj[,colnames(bc_seurat_obj) %in% rownames(bc_seurat_obj_metadata)]

################################### Creating SpaTalk object, and run dec_cci ################################################

################################### Generate pseudo spot data ###############################################################

# create SpaTalk data
obj <- createSpaTalk(st_data = bc_seurat_obj,
                     st_meta = bc_seurat_obj_metadata[,-4],
                     species = "Human",
                     if_st_is_sc = T,
                     spot_max_cell = 10,
                     celltype = bc_seurat_obj_metadata$celltype)

#Find LR Path and Decomposing Cell-type
obj <- find_lr_path(object = obj, lrpairs = lrpairs, pathways = pathways)

for (i in c("stroma"))
{    
    for (j in c('basal_like_1','Tcell','basal_like_2','macrophage','Bcell','mesenchymal','endothelial','luminal_ar'))
    {
        print(c(i,j))
        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
        try(obj <- dec_cci(object=obj, celltype_sender=j, celltype_receiver=i, use_n_cores = 12))
    }
}

for (i in c("basal_like_1"))
{    
    for (j in c('Tcell','basal_like_2','macrophage','Bcell','mesenchymal','endothelial','luminal_ar'))
    {
        print(c(i,j))
        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
        try(obj <- dec_cci(object=obj, celltype_sender=j, celltype_receiver=i, use_n_cores = 12))
    }
}

for (i in c("Tcell"))
{    
    for (j in c('basal_like_2','macrophage','Bcell','mesenchymal','endothelial','luminal_ar'))
    {
        print(c(i,j))
        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
        try(obj <- dec_cci(object=obj, celltype_sender=j, celltype_receiver=i, use_n_cores = 12))
    }
}

for (i in c("basal_like_2"))
{    
    for (j in c('macrophage','Bcell','mesenchymal','endothelial','luminal_ar'))
    {
        print(c(i,j))
        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
        try(obj <- dec_cci(object=obj, celltype_sender=j, celltype_receiver=i, use_n_cores = 12))
    }
}

for (i in c("macrophage"))
{    
    for (j in c('Bcell','mesenchymal','endothelial','luminal_ar'))
    {
        print(c(i,j))
        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
        try(obj <- dec_cci(object=obj, celltype_sender=j, celltype_receiver=i, use_n_cores = 12))
    }
}

for (i in c("Bcell"))
{    
    for (j in c('mesenchymal','endothelial','luminal_ar'))
    {
        print(c(i,j))
        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
        try(obj <- dec_cci(object=obj, celltype_sender=j, celltype_receiver=i, use_n_cores = 12))
    }
}

for (i in c("mesenchymal"))
{    
    for (j in c('endothelial','luminal_ar'))
    {
        print(c(i,j))
        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
        try(obj <- dec_cci(object=obj, celltype_sender=j, celltype_receiver=i, use_n_cores = 12))
    }
}

for (i in c("endothelial"))
{    
    for (j in c('luminal_ar'))
    {
        print(c(i,j))
        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
        try(obj <- dec_cci(object=obj, celltype_sender=j, celltype_receiver=i, use_n_cores = 12))
    }
}


try(obj <- dec_cci(object=obj, celltype_sender="stroma", celltype_receiver="stroma", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="basal_like_1", celltype_receiver="basal_like_1", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Tcell", celltype_receiver="Tcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="basal_like_2", celltype_receiver="basal_like_2", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="macrophage", celltype_receiver="macrophage", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Bcell", celltype_receiver="Bcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="mesenchymal", celltype_receiver="mesenchymal", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="endothelial", celltype_receiver="endothelial", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="luminal_ar", celltype_receiver="luminal_ar", use_n_cores = 12, min_pairs = 3))

write.csv(obj@lrpair,"/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/LR_connectomeDB.csv")
write.csv(obj@tf,"/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/TF_connectomeDB.csv")
saveRDS(obj@cellpair, file = "/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/CellPair_connectomeDB.rds")

Loading required package: ggalluvial

Loading required package: ggplot2

Loading required package: doParallel

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
✔ purrr   1.0.0      
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ purrr::accumulate() masks foreach::accumulate()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
✖ purrr::when()       masks foreach::when()
Attaching SeuratObject


Attaching package: ‘hdf5r’


The following object is masked from ‘package:purrr’:

    flatten_df



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    tra

Checking input data 
Begin to filter lrpairs and pathways 
***Done*** 
Begin to find LR pairs 
Begin to find LR pairs 


#### Simulated Data using stLearn LR database (ConnectomeDB)

In [4]:
################################### Loading the Libraries ###################################################

library(SpaTalk)
library(tidyverse)
library(SeuratData)
library(Seurat)
library(hdf5r)
library(data.table)
library(SeuratDisk)

################################### Reading the ST, SC and Leiden Cluster Data ################################################

sim_seurat_obj = t(read.csv("/afm01/UQ/Q1851/Brad/simulation/spatial_simulation_v2.csv",row.names=1))
sim_seurat_obj_metadata = read.csv("/afm01/UQ/Q1851/Brad/simulation/spatial_simulation_v2_obs.csv",row.names=1)
sim_seurat_obj_metadata <- sim_seurat_obj_metadata[,c("imagerow","imagecol","cell_type")]
colnames(sim_seurat_obj_metadata)[c(1,2,3)] <- c("x","y","celltype")
sim_seurat_obj_metadata <- tibble::rownames_to_column(sim_seurat_obj_metadata, "cell")
sim_seurat_obj_metadata$cell <- paste0("AA",sim_seurat_obj_metadata$cell)
rownames(sim_seurat_obj_metadata) <- sim_seurat_obj_metadata$cell
colnames(sim_seurat_obj) <- rownames(sim_seurat_obj_metadata)

lrpairs <- fread("/home/uqomulay/90days/stLearn_rebuttal/SpaTalk/lrpairs.txt", sep="\t")
lrpairs <- lrpairs[,c("Ligand gene symbol","Receptor gene symbol")]
colnames(lrpairs)[c(1,2)] <- c("ligand","receptor")
lrpairs$species <- rep("Human",2293)

# Checking LR pairs in our data
lrpairs = lrpairs[lrpairs$ligand %in% rownames(sim_seurat_obj) & lrpairs$receptor %in% rownames(sim_seurat_obj)]


################################### Creating SpaTalk object, and run dec_cci ################################################

################################### Generate pseudo spot data ###############################################################

# create SpaTalk data
obj <- createSpaTalk(st_data = sim_seurat_obj,
                     st_meta = sim_seurat_obj_metadata[,-4],
                     species = "Human",
                     if_st_is_sc = T,
                     spot_max_cell = 10,
                     celltype = sim_seurat_obj_metadata$celltype)


#Find LR Path and Decomposing Cell-type
obj <- find_lr_path(object = obj, lrpairs = lrpairs, pathways = pathways)
try(obj <- dec_cci(object=obj, celltype_sender="luminal_ar", celltype_receiver="Bcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="luminal_ar", celltype_receiver="Tcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="luminal_ar", celltype_receiver="mesenchymal", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="luminal_ar", celltype_receiver="stroma", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="luminal_ar", celltype_receiver="luminal_ar", use_n_cores = 12, min_pairs = 3))

try(obj <- dec_cci(object=obj, celltype_sender="Bcell", celltype_receiver="Bcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Bcell", celltype_receiver="Tcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Bcell", celltype_receiver="mesenchymal", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Bcell", celltype_receiver="stroma", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Bcell", celltype_receiver="luminal_ar", use_n_cores = 12, min_pairs = 3))

try(obj <- dec_cci(object=obj, celltype_sender="Tcell", celltype_receiver="Bcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Tcell", celltype_receiver="Tcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Tcell", celltype_receiver="mesenchymal", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Tcell", celltype_receiver="stroma", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="Tcell", celltype_receiver="luminal_ar", use_n_cores = 12, min_pairs = 3))

try(obj <- dec_cci(object=obj, celltype_sender="mesenchymal", celltype_receiver="Bcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="mesenchymal", celltype_receiver="Tcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="mesenchymal", celltype_receiver="mesenchymal", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="mesenchymal", celltype_receiver="stroma", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="mesenchymal", celltype_receiver="luminal_ar", use_n_cores = 12, min_pairs = 3))

try(obj <- dec_cci(object=obj, celltype_sender="stroma", celltype_receiver="Bcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="stroma", celltype_receiver="Tcell", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="stroma", celltype_receiver="mesenchymal", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="stroma", celltype_receiver="stroma", use_n_cores = 12, min_pairs = 3))
try(obj <- dec_cci(object=obj, celltype_sender="stroma", celltype_receiver="luminal_ar", use_n_cores = 12, min_pairs = 3))

#cell_types = ["luminal_ar", "mesenchymal", "Bcell", "Tcell", "stroma"]
#for (i in cell_types)
#{
#    for (j in cell_types)
#    {
#        print(c(i,j))
#        try(obj <- dec_cci(object=obj, celltype_sender=i, celltype_receiver=j, use_n_cores = 12))
#    }
#}

write.csv(obj@lrpair,"/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/LR_connectomeDB_simulation.csv")
write.csv(obj@tf,"/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/TF_connectomeDB_simulation.csv")
saveRDS(obj@cellpair, file = "/home/uqomulay/90days/stLearn_rebuttal/Outputs/SpaTalk/CellPair_connectomeDB_simulation.rds")

Checking input data 
Begin to filter lrpairs and pathways 
***Done*** 
Begin to find LR pairs 
